<a href="https://colab.research.google.com/github/tntnu/20242R0136COSE41600/blob/main/20242R0136COSE41600.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.4 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7

# PointNet

In [ ]:
import os
import numpy as np
import open3d as o3d

def load_point_clouds(data_dirs):
    point_clouds = []
    labels = []  # 객체 탐지에서는 클래스 레이블 필요
    for label, dir_path in enumerate(data_dirs):
        for file_name in os.listdir(dir_path):
            if file_name.endswith('.pcd'):
                file_path = os.path.join(dir_path, file_name)
                pcd = o3d.io.read_point_cloud(file_path)
                point_clouds.append(np.asarray(pcd.points))
                labels.append(label)  # 레이블 예: 폴더 순서에 따라
    return point_clouds, labels

data_dirs = [
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/01_straight_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/02_straight_duck_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/03_straight_crawl/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/04_zigzag_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/05_straight_duck_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/06_straight_crawl/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/07_straight_walk/pcd"
]
point_clouds, labels = load_point_clouds(data_dirs)

In [ ]:
def normalize_point_cloud(point_cloud):
    centroid = np.mean(point_cloud, axis=0)
    point_cloud -= centroid
    max_dist = np.max(np.linalg.norm(point_cloud, axis=1))
    point_cloud /= max_dist
    return point_cloud

def sample_point_cloud(point_cloud, num_points=1024):
    if len(point_cloud) >= num_points:
        indices = np.random.choice(len(point_cloud), num_points, replace=False)
    else:
        indices = np.random.choice(len(point_cloud), num_points, replace=True)
    return point_cloud[indices]

# 정규화 및 샘플링
normalized_point_clouds = [normalize_point_cloud(sample_point_cloud(pc)) for pc in point_clouds]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PointNet(nn.Module):
    def __init__(self, num_classes):
        super(PointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)
        return x


In [ ]:
from torch.utils.data import Dataset, DataLoader

class PointCloudDataset(Dataset):
    def __init__(self, point_clouds, labels):
        self.point_clouds = torch.tensor(point_clouds, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.point_clouds)

    def __getitem__(self, idx):
        return self.point_clouds[idx], self.labels[idx]

dataset = PointCloudDataset(normalized_point_clouds, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PointNet(num_classes=len(data_dirs)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):  # 예: 10 에포크
    model.train()
    total_loss = 0
    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        data = data.permute(0, 2, 1)  # (batch_size, num_points, 3) -> (batch_size, 3, num_points)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")


In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        data = data.permute(0, 2, 1)
        output = model(data)
        pred = output.argmax(dim=1)
        correct += (pred == target).sum().item()
        total += target.size(0)

print(f"Accuracy: {correct / total * 100:.2f}%")


In [ ]:
import open3d as o3d
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.cluster import DBSCAN

# PCD 파일을 로딩하고, Voxel Downsampling 및 클러스터링을 하는 함수
def load_and_preprocess_pcd(file_paths, voxel_size=0.2):
    pcd_list = []
    labels_list = []

    for path in file_paths:
        # 파일을 읽어 포인트 클라우드 로딩
        pcd = o3d.io.read_point_cloud(path)

        # Voxel Downsampling
        pcd_downsampled = pcd.voxel_down_sample(voxel_size=voxel_size)

        # 포인트 클라우드 데이터를 numpy 배열로 변환
        points = np.asarray(pcd_downsampled.points)

        # DBSCAN 클러스터링 (객체를 클러스터로 나눔)
        db = DBSCAN(eps=0.3, min_samples=10).fit(points)
        labels = db.labels_

        # 결과 저장
        pcd_list.append(points)
        labels_list.append(labels)

    return pcd_list, labels_list

# 샘플 경로
file_paths = [
    "C:\\Users\\tntnu\\Desktop\\COSE416_HW1_data_v1\\data\\01_straight_walk\\pcd",
    "C:\\Users\\tntnu\\Desktop\\COSE416_HW1_data_v1\\data\\02_straight_duck_walk\\pcd",
    "C:\\Users\\tntnu\\Desktop\\COSE416_HW1_data_v1\\data\\03_straight_crawl\\pcd",
    "C:\\Users\\tntnu\\Desktop\\COSE416_HW1_data_v1\\data\\04_zigzag_walk\\pcd",
    "C:\\Users\\tntnu\\Desktop\\COSE416_HW1_data_v1\\data\\05_straight_duck_walk\\pcd",
    "C:\\Users\\tntnu\\Desktop\\COSE416_HW1_data_v1\\data\\06_straight_crawl\\pcd",
    "C:\\Users\\tntnu\\Desktop\\COSE416_HW1_data_v1\\data\\07_straight_walk\\pcd"
]

pcd_data, labels_data = load_and_preprocess_pcd(file_paths)

import torch
import torch.nn as nn
import torch.nn.functional as F

# PointNet 모델 정의
class PointNet(nn.Module):
    def __init__(self):
        super(PointNet, self).__init__()

        # MLP 레이어 정의 (포인트별 특성 추출)
        self.mlp1 = nn.Sequential(
            nn.Conv1d(3, 64, 1),
            nn.ReLU(),
            nn.Conv1d(64, 128, 1),
            nn.ReLU(),
            nn.Conv1d(128, 1024, 1),
            nn.ReLU()
        )

        # Max Pooling
        self.pool = nn.MaxPool1d(1024)

        # Fully Connected (FC) 레이어
        self.fc1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 40)  # 객체 종류 40개 (예시)
        )

    def forward(self, x):
        # x는 (batch_size, 3, num_points) 형태
        x = self.mlp1(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

# 모델 생성
model = PointNet()

# PyTorch Dataset 정의
class PointCloudDataset(Dataset):
    def __init__(self, pcd_data, labels_data):
        self.pcd_data = pcd_data
        self.labels_data = labels_data

    def __len__(self):
        return len(self.pcd_data)

    def __getitem__(self, idx):
        points = self.pcd_data[idx]
        labels = self.labels_data[idx]

        # 포인트 클라우드를 (batch_size, 3, num_points) 형태로 변환
        points = torch.tensor(points.T, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.long)

        return points, labels

# 데이터셋 준비
dataset = PointCloudDataset(pcd_data, labels_data)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류 문제로 가정
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for points, labels in dataloader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(points)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 역전파 및 최적화
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}')



# 계층적 샘플링

In [15]:
import os
import random
from collections import defaultdict

def get_files_by_behavior(folders):
    """
    각 행동(폴더)에서 PCD 파일을 분류하여 반환.
    """
    behavior_files = defaultdict(list)
    for folder in folders:
        behavior_name = os.path.basename(os.path.dirname(folder))  # 폴더 이름을 행동 이름으로 사용
        for file_name in os.listdir(folder):
            if file_name.endswith('.pcd'):
                file_path = os.path.join(folder, file_name)
                behavior_files[behavior_name].append(file_path)
    return behavior_files


In [16]:
def stratified_sampling(behavior_files, behavior_ratios):
    """
    각 행동별로 설정된 비율로 데이터를 샘플링.
    """
    sampled_files = []
    for behavior, files in behavior_files.items():
        sample_ratio = behavior_ratios.get(behavior, 0.3)  # 비율이 지정되지 않은 경우 기본값 0.3
        sample_size = int(len(files) * sample_ratio)  # 행동별 샘플링 개수
        sampled_behavior_files = random.sample(files, min(sample_size, len(files)))  # 샘플링
        sampled_files.extend(sampled_behavior_files)  # 결과에 추가
        print(f"[{behavior}] 총 {len(files)}개 중 {len(sampled_behavior_files)}개 샘플링")
    return sampled_files


In [17]:
# 행동별 PCD 파일이 저장된 폴더 리스트
folders = [
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/01_straight_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/02_straight_duck_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/03_straight_crawl/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/04_zigzag_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/05_straight_duck_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/06_straight_crawl/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/07_straight_walk/pcd"
]

In [18]:
# 행동별 샘플링 비율 설정
behavior_ratios = {
    "01_straight_walk": 0.6,
    "02_straight_duck_walk": 0.5,
    "03_straight_crawl": 0.3,
    "04_zigzag_walk": 0.5,
    "05_straight_duck_walk": 0.5,
    "06_straight_crawl": 0.4,
    "07_straight_walk": 0.5
}

In [19]:
# 1. 행동별 파일 리스트 가져오기
behavior_files = get_files_by_behavior(folders)

In [20]:
# 각 행동별 파일 수 출력 (디버깅용)
for behavior, files in behavior_files.items():
    print(f"[{behavior}] 총 파일 수: {len(files)}")

[01_straight_walk] 총 파일 수: 288
[02_straight_duck_walk] 총 파일 수: 549
[03_straight_crawl] 총 파일 수: 1260
[04_zigzag_walk] 총 파일 수: 364
[05_straight_duck_walk] 총 파일 수: 577
[06_straight_crawl] 총 파일 수: 774
[07_straight_walk] 총 파일 수: 443


In [21]:
# 2. 계층적 샘플링 수행
sampled_files = stratified_sampling(behavior_files, behavior_ratios)

print("\n샘플링된 총 파일 수:", len(sampled_files))

[01_straight_walk] 총 288개 중 172개 샘플링
[02_straight_duck_walk] 총 549개 중 274개 샘플링
[03_straight_crawl] 총 1260개 중 378개 샘플링
[04_zigzag_walk] 총 364개 중 182개 샘플링
[05_straight_duck_walk] 총 577개 중 288개 샘플링
[06_straight_crawl] 총 774개 중 309개 샘플링
[07_straight_walk] 총 443개 중 221개 샘플링

샘플링된 총 파일 수: 1824


In [22]:
# 3. 샘플링된 파일 저장

import shutil

def save_sampled_files(sampled_files, output_folder):
    """
    샘플링된 파일을 지정된 폴더에 저장.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_path in sampled_files:
        file_name = os.path.basename(file_path)
        shutil.copy(file_path, os.path.join(output_folder, file_name))

# 3. 샘플링된 파일 저장
output_folder = "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files"
save_sampled_files(sampled_files, output_folder)

# 저장된 파일 목록 출력 (디버깅용)
saved_files = os.listdir(output_folder)
print(f"저장된 파일 목록: {saved_files}")

저장된 파일 목록: ['pcd_000075.pcd', 'pcd_000032.pcd', 'pcd_000158.pcd', 'pcd_000175.pcd', 'pcd_000018.pcd', 'pcd_000058.pcd', 'pcd_000029.pcd', 'pcd_000114.pcd', 'pcd_000017.pcd', 'pcd_000210.pcd', 'pcd_000265.pcd', 'pcd_000267.pcd', 'pcd_000144.pcd', 'pcd_000196.pcd', 'pcd_000272.pcd', 'pcd_000208.pcd', 'pcd_000033.pcd', 'pcd_000065.pcd', 'pcd_000236.pcd', 'pcd_000011.pcd', 'pcd_000143.pcd', 'pcd_000231.pcd', 'pcd_000020.pcd', 'pcd_000170.pcd', 'pcd_000163.pcd', 'pcd_000177.pcd', 'pcd_000007.pcd', 'pcd_000131.pcd', 'pcd_000107.pcd', 'pcd_000035.pcd', 'pcd_000102.pcd', 'pcd_000167.pcd', 'pcd_000148.pcd', 'pcd_000282.pcd', 'pcd_000074.pcd', 'pcd_000193.pcd', 'pcd_000242.pcd', 'pcd_000054.pcd', 'pcd_000076.pcd', 'pcd_000021.pcd', 'pcd_000258.pcd', 'pcd_000142.pcd', 'pcd_000246.pcd', 'pcd_000141.pcd', 'pcd_000049.pcd', 'pcd_000050.pcd', 'pcd_000031.pcd', 'pcd_000108.pcd', 'pcd_000109.pcd', 'pcd_000041.pcd', 'pcd_000186.pcd', 'pcd_000061.pcd', 'pcd_000134.pcd', 'pcd_000201.pcd', 'pcd_000106.pcd'

# 최적의 voxel_size

In [ ]:
def voxel_down_sample(pcd, voxel_size):
    return pcd.voxel_down_sample(voxel_size)

def find_optimal_voxel_size(pcd):
    # 테스트할 voxel 크기 범위 정의
    voxel_sizes = np.arange(0.01, 0.1, 0.01)
    optimal_voxel_size = voxel_sizes[0]
    min_points = len(pcd.points)

    for voxel_size in voxel_sizes:
        downsampled_pcd = voxel_down_sample(pcd, voxel_size)
        if len(downsampled_pcd.points) < min_points:
            min_points = len(downsampled_pcd.points)
            optimal_voxel_size = voxel_size

    return optimal_voxel_size

In [ ]:
# PCD 파일이 있는 디렉토리 경로
pcd_directory = "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files"


In [ ]:
# 디렉토리 내 모든 PCD 파일 목록
pcd_files = [f for f in os.listdir(pcd_directory) if f.endswith('.pcd')]


In [ ]:
import numpy as np

# 각 PCD 파일 처리
for pcd_file in pcd_files:
    pcd_path = os.path.join(pcd_directory, pcd_file)
    pcd = o3d.io.read_point_cloud(pcd_path)

    # 다운샘플링을 위한 최적의 voxel 크기 찾기
    optimal_voxel_size = find_optimal_voxel_size(pcd)

    # 최적의 voxel 크기를 사용하여 포인트 클라우드 다운샘플링
    downsampled_pcd = voxel_down_sample(pcd, optimal_voxel_size)

    # 다운샘플링된 포인트 클라우드 저장
    downsampled_pcd_path = os.path.join(pcd_directory, f"downsampled_{pcd_file}")
    o3d.io.write_point_cloud(downsampled_pcd_path, downsampled_pcd)

    print(f"Processed {pcd_file} with optimal voxel size {optimal_voxel_size}. Downsampled file saved as {downsampled_pcd_path}.")

Processed pcd_000170.pcd with optimal voxel size 0.09. Downsampled file saved as /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/downsampled_pcd_000170.pcd.
Processed pcd_000039.pcd with optimal voxel size 0.09. Downsampled file saved as /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/downsampled_pcd_000039.pcd.
Processed pcd_000135.pcd with optimal voxel size 0.09. Downsampled file saved as /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/downsampled_pcd_000135.pcd.
Processed pcd_000196.pcd with optimal voxel size 0.09. Downsampled file saved as /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/downsampled_pcd_000196.pcd.
Processed pcd_000274.pcd with optimal voxel size 0.09. Downsampled file saved as /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files/dow

# 다운샘플링

In [27]:
import os
import open3d as o3d

def get_pcd_files(input_folder):
    """
    입력 폴더에서 모든 .pcd 파일 경로를 가져옵니다.
    """
    return [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith(".pcd")]

def downsample_and_save(sampled_files, output_folder, voxel_size=0.09):
    """
    샘플링된 파일을 Voxel size로 다운샘플링 후 지정된 폴더에 저장.

    Args:
        sampled_files (list): 다운샘플링할 포인트 클라우드 파일 경로 목록.
        output_folder (str): 다운샘플링된 파일을 저장할 폴더 경로.
        voxel_size (float): Voxel size 값 (기본값: 0.09).
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_path in sampled_files:
        file_name = os.path.basename(file_path)
        output_file_path = os.path.join(output_folder, file_name)

        # 포인트 클라우드 읽기
        pcd = o3d.io.read_point_cloud(file_path)

        # 다운샘플링
        downsampled_pcd = pcd.voxel_down_sample(voxel_size)

        # 다운샘플링된 데이터 저장
        o3d.io.write_point_cloud(output_file_path, downsampled_pcd)
        print(f"다운샘플링 후 저장됨: {output_file_path}")

# 입력 폴더 (샘플링된 파일 폴더)
input_folder = "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/sampled_pcd_files"

# 출력 폴더 (다운샘플링된 파일 저장 경로)
output_folder = "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/downsampled_pcd_files"

# 샘플링된 파일 가져오기
sampled_files = get_pcd_files(input_folder)

# 다운샘플링 및 저장
downsample_and_save(sampled_files, output_folder)

# 저장된 파일 목록 출력 (디버깅용)
saved_files = os.listdir(output_folder)
print(f"저장된 파일 목록: {saved_files}")


다운샘플링 후 저장됨: /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/downsampled_pcd_files/pcd_000075.pcd
다운샘플링 후 저장됨: /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/downsampled_pcd_files/pcd_000032.pcd
다운샘플링 후 저장됨: /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/downsampled_pcd_files/pcd_000158.pcd
다운샘플링 후 저장됨: /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/downsampled_pcd_files/pcd_000175.pcd
다운샘플링 후 저장됨: /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/downsampled_pcd_files/pcd_000018.pcd
다운샘플링 후 저장됨: /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/downsampled_pcd_files/pcd_000058.pcd
다운샘플링 후 저장됨: /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/downsampled_pcd_files/pcd_000029.pcd
다운샘플링 후 저장됨: /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/downsampled_p

# 노이즈 제거(ROR)

In [5]:
downsample_pcd_path = "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/downsampled_pcd_files"

downsample_pcd = o3d.io.read_point_cloud(downsample_pcd_path)

cl, ind = downsample_pcd.remove_radius_outlier(nb_points=6, radius=1.2)
ror_pcd = downsample_pcd.select_by_index(ind)

[Open3D WARNING] Read geometry::PointCloud failed: unknown file extension for /content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/downsampled_pcd_files (format: auto).
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


In [6]:
# RANSAC을 사용하여 평면 추정
plane_model, inliers = ror_pcd.segment_plane(distance_threshold=0.1,
                                             ransac_n=3,
                                             num_iterations=2000)


RuntimeError: [1;31m[Open3D Error] (std::tuple<Eigen::Matrix<double, 4, 1, 0, 4, 1>, std::vector<long unsigned int, std::allocator<long unsigned int> > > open3d::geometry::PointCloud::SegmentPlane(double, int, int, double) const) /root/Open3D/cpp/open3d/geometry/PointCloudSegmentation.cpp:175: There must be at least 'ransac_n' points.
[0;m

In [3]:
import open3d as o3d
import os

# 노이즈 제거 함수: Statistical Outlier Removal 사용
def remove_noise(pcd, nb_neighbors=20, std_ratio=2.0):
    # nb_neighbors: 각 점에 대해 고려할 이웃의 수
    # std_ratio: 표준 편차 범위로 벗어난 점들을 아웃라이어로 간주
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
    return cl

# 각 PCD 파일 처리
for pcd_file in pcd_files:
    pcd_path = os.path.join(pcd_directory, pcd_file)
    pcd = o3d.io.read_point_cloud(pcd_path)

    # 다운샘플링을 위한 최적의 voxel 크기 찾기
    optimal_voxel_size = find_optimal_voxel_size(pcd)

    # 최적의 voxel 크기를 사용하여 포인트 클라우드 다운샘플링
    downsampled_pcd = voxel_down_sample(pcd, optimal_voxel_size)

    # 다운샘플링된 포인트 클라우드에서 노이즈 제거
    denoised_pcd = remove_noise(downsampled_pcd)

    # 노이즈 제거된 포인트 클라우드 저장
    denoised_pcd_path = os.path.join(pcd_directory, f"denoised_{pcd_file}")
    o3d.io.write_point_cloud(denoised_pcd_path, denoised_pcd)

    print(f"Processed {pcd_file} with optimal voxel size {optimal_voxel_size}. Denoised file saved as {denoised_pcd_path}.")


NameError: name 'pcd_files' is not defined

# Transformer

In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import open3d as o3d
from torch.utils.data import DataLoader, Dataset

class LiDARPointCloudDataset(Dataset):
    def __init__(self, folders, num_points=4096):
        self.file_paths = []
        self.labels = []
        self.num_points = num_points

        # 각 폴더를 순회하며 파일 경로와 레이블 저장
        for label, folder in enumerate(folders):
            for file_name in os.listdir(folder):
                if file_name.endswith('.pcd'):
                    self.file_paths.append(os.path.join(folder, file_name))
                    self.labels.append(label)

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        # Load .pcd file
        pcd = o3d.io.read_point_cloud(self.file_paths[idx])
        points = np.asarray(pcd.points, dtype=np.float32)

        # Normalize points to [-1, 1]
        points -= np.mean(points, axis=0)
        points /= np.max(np.linalg.norm(points, axis=1))

        # Random sampling
        if points.shape[0] > self.num_points:
            indices = np.random.choice(points.shape[0], self.num_points, replace=False)
        else:
            indices = np.random.choice(points.shape[0], self.num_points, replace=True)
        points = points[indices]

        return torch.tensor(points, dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

# 행동별 PCD 파일이 저장된 폴더 리스트
folders = [
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/01_straight_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/02_straight_duck_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/03_straight_crawl/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/04_zigzag_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/05_straight_duck_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/06_straight_crawl/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/07_straight_walk/pcd"
]

# 데이터셋 및 데이터로더
dataset = LiDARPointCloudDataset(folders)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


In [6]:
# --- 2. Transformer 기반 모델 정의 ---
class TransformerModel(nn.Module):
    def __init__(self, input_dim=3, embed_dim=128, num_heads=4, num_classes=7):  # num_classes를 7로 설정
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, embed_dim)
        self.transformer = nn.Transformer(
            d_model=embed_dim,
            nhead=num_heads,
            num_encoder_layers=6,
            num_decoder_layers=6,
            dim_feedforward=512,
            dropout=0.1,
        )
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        # x: [batch_size, num_points, input_dim]
        batch_size, num_points, _ = x.size()
        x = self.embedding(x)  # [batch_size, num_points, embed_dim]
        x = x.permute(1, 0, 2)  # Transformer expects [num_points, batch_size, embed_dim]
        x = self.transformer(x, x)  # Self-attention
        x = x.mean(dim=0)  # Pooling
        x = self.classifier(x)  # [batch_size, num_classes]
        return x


In [7]:
# --- 3. 학습 루프 ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        points, labels = batch
        points, labels = points.to(device), labels.to(device)  # points와 labels 모두 device로 이동

        optimizer.zero_grad()
        outputs = model(points)  # 모델에 입력
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 파라미터 업데이트

        total_loss += loss.item()  # 손실 누적
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# --- 4. 결과 시각화 ---
def visualize_point_cloud(points, title="Point Cloud"):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    o3d.visualization.draw_geometries([pcd], window_name=title)

In [ ]:
# 테스트 데이터 시각화
test_data, _ = dataset[0]  # 첫 번째 샘플 데이터와 레이블
visualize_point_cloud(test_data.numpy(), title="Input Point Cloud")

# .

In [ ]:
# 시각화에 필요한 라이브러리 불러오기
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

# pcd 파일 불러오기, 필요에 맞게 경로 수정
file_path = "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_tutorial/COSE416_HW1_tutorial/test_data/1727320101-665925967.pcd"
# PCD 파일 읽기
original_pcd = o3d.io.read_point_cloud(file_path)

In [ ]:
# Voxel Downsampling 수행
voxel_size = 0.5  # 필요에 따라 voxel 크기를 조정하세요. 0.2 -> 0.5로 수정
downsample_pcd = original_pcd.voxel_down_sample(voxel_size=voxel_size)

In [ ]:
# Radius Outlier Removal (ROR) 적용
cl, ind = downsample_pcd.remove_radius_outlier(nb_points=6, radius=1.2)
ror_pcd = downsample_pcd.select_by_index(ind)

In [ ]:
# RANSAC을 사용하여 평면 추정
plane_model, inliers = ror_pcd.segment_plane(distance_threshold=0.1,
                                             ransac_n=3,
                                             num_iterations=2000)

In [ ]:
# 도로에 속하지 않는 포인트 (outliers) 추출
final_point = ror_pcd.select_by_index(inliers, invert=True)

In [ ]:
# DBSCAN 클러스터링 적용
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(final_point.cluster_dbscan(eps=0.5, min_points=5, print_progress=True)) # eps = 0.3 -> 0.5, min_points = 10 -> 5 값 수정 후 클러스터링 성공

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 350


In [ ]:
# 노이즈 포인트는 검정색, 클러스터 포인트는 파란색으로 지정
colors = np.zeros((len(labels), 3))  # 기본 검정색 (노이즈)
colors[labels >= 0] = [0, 0, 1]  # 파란색으로 지정

final_point.colors = o3d.utility.Vector3dVector(colors)

In [ ]:
# 포인트 클라우드를 시각화하는 함수
def visualize_point_cloud(pcd, window_name="Point Cloud", point_size=1.0):
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name=window_name)
    vis.add_geometry(pcd)
    vis.get_render_option().point_size = point_size
    vis.run()
    vis.destroy_window()

In [ ]:
# 시각화 (포인트 크기를 원하는 크기로 조절 가능)
visualize_point_cloud(final_point, point_size=2.0)

# 세션 다운 됨

[Open3D WARNING] Failed to initialize GLFW


AttributeError: 'NoneType' object has no attribute 'point_size'

# 다운샘플링+노이즈제거+클러스터링

In [8]:
# 행동별 PCD 파일이 저장된 폴더 리스트
folders = [
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/01_straight_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/02_straight_duck_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/03_straight_crawl/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/04_zigzag_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/05_straight_duck_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/06_straight_crawl/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/07_straight_walk/pcd"
]

In [10]:
import os
import open3d as o3d
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt


# 다운샘플링 함수: Voxel Grid 필터링
def voxel_down_sample(pcd, voxel_size):
    return pcd.voxel_down_sample(voxel_size)


# 노이즈 제거 함수: Statistical Outlier Removal
def remove_noise(pcd, nb_neighbors=20, std_ratio=2.0):
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
    return cl


# DBSCAN 클러스터링 함수
def apply_dbscan(pcd, eps=0.5, min_samples=10):
    points = np.asarray(pcd.points)
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(points)
    return db.labels_


# LiDAR 데이터셋 클래스
class LiDARPointCloudDataset(Dataset):
    def __init__(self, file_paths, num_points=4096):
        self.file_paths = file_paths
        self.num_points = num_points

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        pcd_path = self.file_paths[idx]
        pcd = o3d.io.read_point_cloud(pcd_path)
        points = np.asarray(pcd.points, dtype=np.float32)

        # 포인트 수가 부족하면 샘플링, 초과하면 무작위 샘플링
        if points.shape[0] > self.num_points:
            indices = np.random.choice(points.shape[0], self.num_points, replace=False)
        else:
            indices = np.random.choice(points.shape[0], self.num_points, replace=True)

        points = points[indices]

        # 텐서로 반환
        return torch.tensor(points, dtype=torch.float32)


# Transformer 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_dim=3, embed_dim=128, num_heads=4, num_classes=10):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, embed_dim)
        self.transformer = nn.Transformer(
            d_model=embed_dim,
            nhead=num_heads,
            num_encoder_layers=6,
            num_decoder_layers=6,
            dim_feedforward=512,
            dropout=0.1,
        )
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        batch_size, num_points, _ = x.size()
        x = self.embedding(x)  # [batch_size, num_points, embed_dim]
        x = x.permute(1, 0, 2)  # Transformer expects [num_points, batch_size, embed_dim]
        x = self.transformer(x, x)  # Self-attention
        x = x.mean(dim=0)  # Pooling
        x = self.classifier(x)  # [batch_size, num_classes]
        return x


# 포인트 클라우드 전처리 및 모델 학습
def process_and_train(pcd_files, num_epochs=10, batch_size=16, lr=1e-4):
    dataset = LiDARPointCloudDataset(pcd_files)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # 모델 초기화
    model = TransformerModel(input_dim=3, embed_dim=128, num_heads=4, num_classes=10).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in dataloader:
            batch = batch.cuda()

            # Dummy labels (클러스터링 결과로 레이블을 대체할 수 있음)
            labels = torch.randint(0, 10, (batch.size(0),)).cuda()

            optimizer.zero_grad()
            outputs = model(batch)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")





In [11]:
# LiDAR 데이터셋 준비
folders = [
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/01_straight_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/02_straight_duck_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/03_straight_crawl/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/04_zigzag_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/05_straight_duck_walk/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/06_straight_crawl/pcd",
    "/content/drive/MyDrive/ColabNotebooks/20242R0136COSE41600/COSE416_HW1_data_v1/data/07_straight_walk/pcd"
]



In [12]:
# 각 폴더에 있는 PCD 파일을 로드
pcd_files = []
for folder in folders:
    pcd_files += [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.pcd')]



In [14]:
# 각 PCD 파일 처리 (다운샘플링, 노이즈 제거, 클러스터링)
for pcd_file in pcd_files:
    pcd = o3d.io.read_point_cloud(pcd_file)

    # 다운샘플링
    optimal_voxel_size = 0.05  # 예시로 고정된 voxel size 사용
    downsampled_pcd = voxel_down_sample(pcd, optimal_voxel_size)

    # 노이즈 제거
    denoised_pcd = remove_noise(downsampled_pcd)

    # DBSCAN 클러스터링
    cluster_labels = apply_dbscan(denoised_pcd)

    # 클러스터링 결과를 포인트 클라우드에 색상으로 표시
    points = np.asarray(denoised_pcd.points)
    colors = np.array([plt.cm.get_cmap('tab20')(label / (max(cluster_labels) + 1))[:3] for label in cluster_labels])
    denoised_pcd.colors = o3d.utility.Vector3dVector(colors)




<ipython-input-14-81781815aead>:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = np.array([plt.cm.get_cmap('tab20')(label / (max(cluster_labels) + 1))[:3] for label in cluster_labels])
<ipython-input-14-81781815aead>:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = np.array([plt.cm.get_cmap('tab20')(label / (max(cluster_labels) + 1))[:3] for label in cluster_labels])
<ipython-input-14-81781815aead>:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = 

KeyboardInterrupt: 

In [ ]:
# 모델 학습
process_and_train(pcd_files)